# Algorytm genetyczny
## Anna Urbala

In [5]:
import numpy as np
import random
import math
import copy

In [38]:
class GenAlg:
    def __init__(self, fun, n_var, n_population=1000, rand_min=-10, rand_max=10):
        self.fun = fun
        self.n_var = n_var
        self.n_population = n_population
        self.population = np.random.uniform(rand_min, rand_max, size=(n_population, n_var))
        
    def iterate(self, n_iterations=100):
        for i in range(n_iterations):
            mutated = self.mutation()
            children = self.crossover()
            current_population = np.concatenate((self.population, mutated, children))
            evaluated = self.evaluate(current_population)
            new_population = self.selection(current_population, evaluated)
            self.population = new_population
    
    def mutation(self):
        p = np.random.permutation(self.n_population)
        mutated_size = math.floor(self.n_population*0.2)
        return self.population[p][0:mutated_size] + np.random.normal(0, 1, size=(mutated_size, self.n_var))
    
    def crossover(self):
        p = np.random.permutation(self.n_population)
        parents_size = math.floor(self.n_population*0.7)
        parents = self.population[p][0:parents_size]
        children = np.array([self.cross(parents[i], parents[i+1]) for i in range(0, parents_size-1, 2)])
        return children.reshape(len(children) * 2, self.n_var)
        
    @staticmethod
    def cross(parent1, parent2):
        i = random.randint(0, len(parent1)-1)
        child1 = np.concatenate((parent1[0:i],parent2[i:]))
        child2 = np.concatenate((parent2[0:i],parent1[i:]))
        return [child1, child2]
    
    def evaluate(self, population):
        outputs = list(map(self.fun, population))
        population_min = min(outputs)
        return outputs - population_min + 0.01
    
    def selection(self, population, values):
        new_population = copy.deepcopy(population)
        to_remove = len(population) - self.n_population
        for i in range(to_remove):
            idx = random.choices(range(len(values)), weights=values, k=1)
            new_population = np.delete(new_population, idx, axis=0)
            values = np.delete(values, idx)
        return new_population
    
    def get_min(self):
        evaluated = self.evaluate(self.population)
        return self.population[evaluated == min(evaluated)][0]
        

## Funkcja $x^2+y^2+2z^2$

In [47]:
def r3(elem):
    x,y,z = elem
    return x**2 + y**2 + 2*z**2

In [54]:
np.random.seed(2137)
genalg = GenAlg(r3, 3, 100)
genalg.iterate(100)

In [55]:
opt = genalg.get_min()
print(opt)
print(r3(opt))

[-0.02332001 -0.06248066  0.00183841]
0.004454415826340502


Już dla 100 iteracji i populacji wielkości 100 dostaliśmy sensowne minimum, bliskie prawdziwego.

## Funkcja Rastrigina
### $f(x)=10n+\sum_{i=1}^{n}[x_i^2-10\cos{(2\pi x_i)}]$

Dwywumiarowa:
![Rastrigin](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8b/Rastrigin_function.png/300px-Rastrigin_function.png)

In [56]:
def rastrigin(elem):
    return 50 + sum([x**2 - 10*math.cos(2*math.pi*x) for x in elem])

In [59]:
np.random.seed(2137)
genalg = GenAlg(rastrigin, 5, 100)
genalg.iterate(1000)

In [60]:
opt = genalg.get_min()
print(opt)
print(rastrigin(opt))

[0.06072098 0.0012842  0.02828824 0.01762384 0.05346881]
1.5048198425634212


Algorytm poradził sobie dość dobrze. Minimum wypada w punkcie (0,0,0,0,0) i wynosi 0. Co prawda wartość wyszła większa niż w poprzednim podpunkcie, ale funkcja jest bardziej skomplikowana i ma wiele minimów lokalnych.